## 인공신경망

### 단일 퍼셉트론

##### 인공신경망
- 1950년 부터 연구된 인공신경망(Artificial Neural Network: ANN)에 기초
- 생물학적 신경망에서 영감을 받아 만들어진 계산구조

    <img src="https://framerusercontent.com/images/foP4Gj0q0SLo6Z1VVGRUe63abk.jpg" width="800" height="400">

##### Perceptron
- 1957년 Frank Resonblatt이 고안한 인공신경망 구조

    <img src="https://raw.githubusercontent.com/hugoMGSung/study-pytorch/refs/heads/main/images/torch0011.png" width="800">


    - Inputs - 실수값을 갖는 벡터
    - Weights - 가중치 (선형결합에 사용될 값)
    - Neurone 
        - 뇌 세포의 이름과 동일. 직접 입력값을 처리하는 영역. 입력값에 특정 가중치를 적용한 것들을 모두 합산
        - 뉴런의 활성화 함수로 전달
        - bias(편향)는 뉴런이 모든 입력값이 0일때에도 출력을 생성할 수 있게 해주는 역할
    - Activation - 뉴런에서 전달받은 값에 비선형을 학습하고 복잡한 패턴을 표현할 수 있게 해줌
    - Output - -1 ~ 1 사이의 값을 출력


##### 입력된 X에 대한 출력 Y
- X -> w -> +b -> Activation function -> Y

#### OR 분류문제
- 간단한 논리 연산자를 학습하는 딥러닝 문제 중 하나. OR 논리 게이트를 모델로 학습시키는 것
- OR 게이트는 두개의 입력값 중 하나라도 True(1)이면 출력이 1이 되는 연산

|input x1|input x2|output y(OR)|
|---:|---:|---:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|1|

##### 모델 구조

$$ z = w_{1}x_{1} + w_{2}x_{2} + b $$ 
$$ y = Activation(z) $$

- w1, w2 는 가중치, b 는 편향, Activation 은 Sigmoid 등의 할성화 함수
- 분류문제므로 손실함수로 이진 교차 엔트로피를 주로 사용
- 학습목표는 가중치와 편향을 학습하여 출력y가 OR 논리를 만족하도록 만드는 것
- Backpropagation 역전파와 Graident Descent 경사하강법으로 학습

### 단일 퍼셉트론의 문제점

#### OR문제 학습

In [1]:
import torch
import torch.nn as nn

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

##### OR값

In [6]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[1]]).to(device)

##### 훈련

In [7]:
linear = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

# 비용함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device) # Binary Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=1)

# 10_000번
for epoch in range(10000):
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:04d}, Cost: {cost.item():.9f}')

Epoch: 0000, Cost: 0.820362329
Epoch: 0100, Cost: 0.093902916
Epoch: 0200, Cost: 0.048203304
Epoch: 0300, Cost: 0.032028854
Epoch: 0400, Cost: 0.023880810
Epoch: 0500, Cost: 0.019001313
Epoch: 0600, Cost: 0.015761431
Epoch: 0700, Cost: 0.013457287
Epoch: 0800, Cost: 0.011736300
Epoch: 0900, Cost: 0.010402864
Epoch: 1000, Cost: 0.009339727
Epoch: 1100, Cost: 0.008472646
Epoch: 1200, Cost: 0.007752014
Epoch: 1300, Cost: 0.007143807
Epoch: 1400, Cost: 0.006623670
Epoch: 1500, Cost: 0.006173852
Epoch: 1600, Cost: 0.005780944
Epoch: 1700, Cost: 0.005434918
Epoch: 1800, Cost: 0.005127794
Epoch: 1900, Cost: 0.004853448
Epoch: 2000, Cost: 0.004606802
Epoch: 2100, Cost: 0.004383982
Epoch: 2200, Cost: 0.004181684
Epoch: 2300, Cost: 0.003997146
Epoch: 2400, Cost: 0.003828149
Epoch: 2500, Cost: 0.003672857
Epoch: 2600, Cost: 0.003529606
Epoch: 2700, Cost: 0.003397123
Epoch: 2800, Cost: 0.003274194
Epoch: 2900, Cost: 0.003159783
Epoch: 3000, Cost: 0.003053137
Epoch: 3100, Cost: 0.002953388
Epoch: 3

##### 테스트

In [14]:
with torch.no_grad():
    hypothesis = model(X)
    predict = (hypothesis > 0.5).float()
    accuracy = (predict == Y).float().mean()
    print(f"모델 출력값(Hypothesis): {hypothesis.detach().cpu().numpy()}")
    print(f'모델 예측값(Predict): {predict.detach().cpu().numpy()}')
    print(f'실제값(Y): {Y.detach().cpu().numpy()}')
    print(f'정확도(Accuracy): {accuracy.item()}')


모델 출력값(Hypothesis): [[0.00201262]
 [0.99919504]
 [0.99919504]
 [1.        ]]
모델 예측값(Predict): [[0.]
 [1.]
 [1.]
 [1.]]
실제값(Y): [[0.]
 [1.]
 [1.]
 [1.]]
정확도(Accuracy): 1.0


##### 여기까지는 별문제가 없어 보임!

#### 한계
- Non-linear 문제가 존재

    <img src="https://raw.githubusercontent.com/hugoMGSung/study-pytorch/refs/heads/main/images/torch0012.png" width="800">

- 예: XOR 문제

    [참조링크](https://medium.com/@lmpo/a-brief-history-of-ai-with-deep-learning-26f7948bc87b)

### 다중계층 퍼셉트론

#### 해결법
- 두 단계로 분류를 진행하면 해결 가능
- 단일 퍼셉트론을 다중으로 겹침으로써 해결가능한 문제의 범주를 확대

##### XOR 값

In [15]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

In [16]:
mapping = nn.Linear(2, 2, bias=True) ## 입력값 두개를 받아서 2개의 출력으로
classfier = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(mapping, sigmoid, classfier, sigmoid).to(device)

# 비용함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device) # Binary Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=1)

# 10_000번
for epoch in range(10000):
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:04d}, Cost: {cost.item():.9f}')

Epoch: 0000, Cost: 0.724791050
Epoch: 0100, Cost: 0.693008661
Epoch: 0200, Cost: 0.692750096
Epoch: 0300, Cost: 0.692028165
Epoch: 0400, Cost: 0.687997580
Epoch: 0500, Cost: 0.647870660
Epoch: 0600, Cost: 0.519063652
Epoch: 0700, Cost: 0.205190748
Epoch: 0800, Cost: 0.078483582
Epoch: 0900, Cost: 0.045516428
Epoch: 1000, Cost: 0.031572908
Epoch: 1100, Cost: 0.024023829
Epoch: 1200, Cost: 0.019329097
Epoch: 1300, Cost: 0.016140645
Epoch: 1400, Cost: 0.013839476
Epoch: 1500, Cost: 0.012103309
Epoch: 1600, Cost: 0.010748316
Epoch: 1700, Cost: 0.009662222
Epoch: 1800, Cost: 0.008772782
Epoch: 1900, Cost: 0.008031387
Epoch: 2000, Cost: 0.007404077
Epoch: 2100, Cost: 0.006866590
Epoch: 2200, Cost: 0.006401021
Epoch: 2300, Cost: 0.005993940
Epoch: 2400, Cost: 0.005635011
Epoch: 2500, Cost: 0.005316248
Epoch: 2600, Cost: 0.005031239
Epoch: 2700, Cost: 0.004774994
Epoch: 2800, Cost: 0.004543333
Epoch: 2900, Cost: 0.004332924
Epoch: 3000, Cost: 0.004140973
Epoch: 3100, Cost: 0.003965179
Epoch: 3

##### 테스트

In [17]:
with torch.no_grad():
    hypothesis = model(X)
    predict = (hypothesis > 0.5).float()
    accuracy = (predict == Y).float().mean()
    print(f"모델 출력값(Hypothesis): {hypothesis.detach().cpu().numpy()}")
    print(f'모델 예측값(Predict): {predict.detach().cpu().numpy()}')
    print(f'실제값(Y): {Y.detach().cpu().numpy()}')
    print(f'정확도(Accuracy): {accuracy.item()}')

모델 출력값(Hypothesis): [[9.5541280e-04]
 [9.9868053e-01]
 [9.9911124e-01]
 [8.3470478e-04]]
모델 예측값(Predict): [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y): [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy): 1.0
